In [121]:
import json
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

#read-in and preprocess
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
credits = credits.drop(columns='title')

#combine and preprocess
fulldf = pd.concat([credits,movies],axis=1)
# fulldf.info()
fulldf = fulldf.drop(columns=['id','overview','tagline','homepage'])
fulldf['profit'] = fulldf['revenue'] - fulldf['budget']


fulldf.query("(original_title != title)")[['title','original_title']] #comparing original title and title
# a = fulldf.isnull().any()
# print(a[a==True])

fulldf[fulldf['runtime'].isnull()]
fulldf.at[2656,'runtime'] =113 
fulldf['runtime'][4140] = 91
fulldf['release_date'] = fulldf['release_date'].fillna('2014-06-01')
fulldf['release_date'] = pd.to_datetime(fulldf['release_date'],format='%Y-%m-%d')


fulldf['genres'] = fulldf['genres'].apply(json.loads)
def decode(columns):
    z = []
    for i in columns:
        z.append(i['name'])
    return ','.join(z)
fulldf['genres'] = fulldf['genres'].apply(decode)

fulldf['release_date'] = pd.to_datetime(fulldf['release_date']).dt.year
columns = {'release_date':'year'}
fulldf.rename(columns=columns,inplace=True)
fulldf['year'].apply(int).head()
fulldf.columns

genre_list = set()
for i in fulldf['genres'].str.split(','):
    genre_list = set().union(i,genre_list)
    genre_list = list(genre_list)
    
genre_list.remove('')

for genre in genre_list:
    fulldf[genre] = fulldf['genres'].str.contains(genre).apply(lambda x:1 if x else 0)
genre_year = fulldf.loc[:,genre_list]
genre_year.index = fulldf['year']
genresdf = genre_year.groupby('year').sum()
genresdfSum = genresdf.sum(axis=0).sort_values(ascending=False)

genresdfSum

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Drama              2297
Comedy             1722
Thriller           1274
Action             1154
Romance             894
Adventure           790
Crime               696
Science Fiction     535
Horror              519
Family              513
Fantasy             424
Mystery             348
Animation           234
History             197
Music               185
War                 144
Documentary         110
Western              82
Foreign              34
TV Movie              8
dtype: int64

In [123]:
import plotly.express as px

# fig = plt.figure(figsize=(12,8))
# ax1 = plt.subplot(111)
# rects = genresdfSum.sort_values(ascending=True).plot(kind='barh',label='genres')
# plt.title('各种电影类型数据统计图')
# plt.xlabel('电影数量',fontsize=15)
# plt.ylabel('电影类型',fontsize=15)
# plt.show()

fig = px.bar(genresdfSum,labels={'value':'Number of Movies','index':'Genre'},title="Movies by Genre")
fig.show()

fig2 = px.pie(genresdfSum,values=genresdfSum.values, names=genresdfSum.index, title='Movies by Genre')
fig2.show()